# Stock NeurIPS2018 Part 1. Data
This series is a reproduction of paper *the process in the paper Practical Deep Reinforcement Learning Approach for Stock Trading*. 

This is the first part of the NeurIPS2018 series, introducing how to use FinRL to fetch and process data that we need for ML/RL trading.

Other demos can be found at the repo of [FinRL-Tutorials]((https://github.com/AI4Finance-Foundation/FinRL-Tutorials)).

# Part 1. Install Packages

In [ ]:
## install finrl library
!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git

In [1]:
import pandas as pd
import numpy as np
import datetime
import yfinance as yf

from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl import config_tickers
from finrl.config import INDICATORS

import itertools

C:\Users\aryam\AppData\Local\Temp\ipykernel_45732\1185721934.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was too old on your system - pyarrow 10.0.1 is the current minimum supported version as of this release.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# Part 2. Fetch data

[yfinance](https://github.com/ranaroussi/yfinance) is an open-source library that provides APIs fetching historical data form Yahoo Finance. In FinRL, we have a class called [YahooDownloader](https://github.com/AI4Finance-Foundation/FinRL/blob/master/finrl/meta/preprocessor/yahoodownloader.py) that use yfinance to fetch data from Yahoo Finance.

**OHLCV**: Data downloaded are in the form of OHLCV, corresponding to **open, high, low, close, volume,** respectively. OHLCV is important because they contain most of numerical information of a stock in time series. From OHLCV, traders can get further judgement and prediction like the momentum, people's interest, market trends, etc.

## Data for a single ticker

Here we provide two ways to fetch data with single ticker, let's take Apple Inc. (AAPL) as an example.

### Using yfinance

In [2]:
ril_df_yf = yf.download(tickers = "RELIANCE.NS", start='2020-01-01', end='2020-01-31')

c:\Program Files\Python310\lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*********************100%%**********************]  1 of 1 completed


In [3]:
ril_df_yf.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-01-01,1387.957031,1396.277466,1376.527954,1380.276611,1360.645142,7002234
2020-01-02,1382.471069,1408.941040,1382.471069,1403.775024,1383.809448,8855158
2020-01-03,1401.671997,1409.581055,1392.528687,1405.466553,1385.476807,10492349
2020-01-06,1389.785767,1397.008911,1369.670410,1372.870605,1353.344482,12259588
2020-01-07,1388.871338,1403.043579,1383.842529,1393.991699,1374.165161,8341811


### Using FinRL

In FinRL's YahooDownloader, we modified the data frame to the form that convenient for further data processing process. We use adjusted close price instead of close price, and add a column representing the day of a week (0-4 corresponding to Monday-Friday).

In [4]:
ril_df_finrl = YahooDownloader(start_date = '2020-01-01',
                                end_date = '2020-01-31',
                                ticker_list = ['RELIANCE.NS']).fetch_data()

c:\Program Files\Python310\lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*********************100%%**********************]  1 of 1 completed

Shape of DataFrame:  (22, 8)


In [5]:
ril_df_finrl.head()

,date,open,high,low,close,volume,tic,day
0,2020-01-01,1387.957031,1396.277466,1376.527954,1360.645142,7002234,RELIANCE.NS,2
1,2020-01-02,1382.471069,1408.941040,1382.471069,1383.809448,8855158,RELIANCE.NS,3
2,2020-01-03,1401.671997,1409.581055,1392.528687,1385.476807,10492349,RELIANCE.NS,4
3,2020-01-06,1389.785767,1397.008911,1369.670410,1353.344482,12259588,RELIANCE.NS,0
4,2020-01-07,1388.871338,1403.043579,1383.842529,1374.165161,8341811,RELIANCE.NS,1


## Data for the chosen tickers

In [6]:
config_tickers.SENSEX_TICKER

['ASIANPAINT.NS',
 'AXISBANK.NS',
 'BAJFINANCE.NS',
 'BAJAJFINSV.NS',
 'BHARTIARTL.NS',
 'HCLTECH.NS',
 'HDFCBANK.NS',
 'HINDUNILVR.NS',
 'ICICIBANK.NS',
 'INDUSINDBK.NS',
 'INFY.NS',
 'JSWSTEEL.NS',
 'KOTAKBANK.NS',
 'LT.NS',
 'M&M.NS',
 'MARUTI.NS',
 'NESTLEIND.NS',
 'NTPC.NS',
 'POWERGRID.NS',
 'RELIANCE.NS',
 'SBIN.NS',
 'SUNPHARMA.NS',
 'TCS.NS',
 'TATAMOTORS.NS',
 'TATASTEEL.NS',
 'TECHM.NS',
 'TITAN.NS',
 'ULTRACEMCO.NS',
 'WIPRO.NS']

In [7]:
TRAIN_START_DATE = '2011-01-01'
TRAIN_END_DATE = '2020-07-01'
TRADE_START_DATE = '2020-07-01'
TRADE_END_DATE = '2023-10-29'

In [8]:
df_raw = YahooDownloader(start_date = TRAIN_START_DATE,
                     end_date = TRADE_END_DATE,
                     ticker_list = config_tickers.SENSEX_TICKER).fetch_data()

c:\Program Files\Python310\lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*********************100%%**********************]  1 of 1 completed
c:\Program Files\Python310\lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*********************100%%**********************]  1 of 1 completed
c:\Program Files\Python310\lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*********************100%%*******************

Shape of DataFrame:  (91698, 8)


In [9]:
df_raw.head()

,date,open,high,low,close,volume,tic,day
0,2011-01-03,289.799988,293.739990,286.005005,259.076111,454300,ASIANPAINT.NS,0
1,2011-01-03,273.000000,274.399994,268.459991,251.916641,5266100,AXISBANK.NS,0
2,2011-01-03,45.497864,46.357517,45.478214,45.569965,120767,BAJAJFINSV.NS,0
3,2011-01-03,69.832405,69.832405,68.341026,63.384369,129932,BAJFINANCE.NS,0
4,2011-01-03,325.263733,328.958893,319.721008,303.569275,3283719,BHARTIARTL.NS,0


# Part 3: Preprocess Data
We need to check for missing data and do feature engineering to convert the data point into a state.
* **Adding technical indicators**. In practical trading, various information needs to be taken into account, such as historical prices, current holding shares, technical indicators, etc. Here, we demonstrate two trend-following technical indicators: MACD and RSI.
* **Adding turbulence index**. Risk-aversion reflects whether an investor prefers to protect the capital. It also influences one's trading strategy when facing different market volatility level. To control the risk in a worst-case scenario, such as financial crisis of 2007–2008, FinRL employs the turbulence index that measures extreme fluctuation of asset price.

Hear let's take **MACD** as an example. Moving average convergence/divergence (MACD) is one of the most commonly used indicator showing bull and bear market. Its calculation is based on EMA (Exponential Moving Average indicator, measuring trend direction over a period of time.)

In [10]:
fe = FeatureEngineer(use_technical_indicator=True,
                     tech_indicator_list = INDICATORS,
                     use_vix=True,
                     use_turbulence=True,
                     user_defined_feature = False)

processed = fe.preprocess_data(df_raw)

Successfully added technical indicators


c:\Program Files\Python310\lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*********************100%%**********************]  1 of 1 completed


Shape of DataFrame:  (3226, 8)
Successfully added vix
Successfully added turbulence index


In [11]:
list_ticker = processed["tic"].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
combination = list(itertools.product(list_date,list_ticker))

processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])

processed_full = processed_full.fillna(0)

In [12]:
processed_full.head()

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2011-01-03,ASIANPAINT.NS,289.799988,293.739990,286.005005,259.076111,454300.0,0.0,0.0,262.236018,252.459783,0.0,-66.666667,100.0,259.076111,259.076111,17.610001,0.0
1,2011-01-03,AXISBANK.NS,273.000000,274.399994,268.459991,251.916641,5266100.0,0.0,0.0,262.236018,252.459783,0.0,-66.666667,100.0,251.916641,251.916641,17.610001,0.0
2,2011-01-03,BAJAJFINSV.NS,45.497864,46.357517,45.478214,45.569965,120767.0,0.0,0.0,262.236018,252.459783,0.0,-66.666667,100.0,45.569965,45.569965,17.610001,0.0
3,2011-01-03,BAJFINANCE.NS,69.832405,69.832405,68.341026,63.384369,129932.0,0.0,0.0,262.236018,252.459783,0.0,-66.666667,100.0,63.384369,63.384369,17.610001,0.0
4,2011-01-03,BHARTIARTL.NS,325.263733,328.958893,319.721008,303.569275,3283719.0,0.0,0.0,262.236018,252.459783,0.0,-66.666667,100.0,303.569275,303.569275,17.610001,0.0


# Part 4: Save the Data

### Split the data for training and trading

In [13]:
train = data_split(processed_full, TRAIN_START_DATE,TRAIN_END_DATE)
trade = data_split(processed_full, TRADE_START_DATE,TRADE_END_DATE)
print(len(train))
print(len(trade))

65714
23200


### Save data to csv file

For Colab users, you can open the virtual directory in colab and manually download the files.

For users running on your local environment, the csv files should be at the same directory of this notebook.

In [18]:
train.to_csv('train_data.csv')
trade.to_csv('trade_data.csv')